# Feature Selection Notebook for Dengue Transfer Learning Project

### Dataset
Dengue ML datasets track environmental and temporal factors influencing Aedes mosquito breeding and virus transmission in tropical regions like San Juan and Iquitos.

- #### Temporal Features
    - **city**: Location identifier (e.g., 'sj' for San Juan, 'iq' for Iquitos)—captures city-specific mosquito/dengue patterns.
    - **year, weekofyear, week_start_date**: Time granularity for seasonality; dengue peaks during rainy seasons (weekofyear critical for lagged effects).

- #### Vegetation Indices (NDVI)
    - **ndvi_ne, ndvi_nw, ndvi_se, ndvi_sw**: Normalized Difference Vegetation Index by city quadrant. Higher NDVI indicates lush vegetation providing mosquito shade/breeding sites; key for Aedes habitat detection via satellite.

- #### Precipitation \& Water
    - **precipitation_amt_mm**: Rainfall amount—creates standing water breeding sites.
    - **reanalysis_precip_amt_kg_per_m2, reanalysis_sat_precip_amt_mm**: Reanalysis (modeled) precipitation variants confirming observed rain.
    - **station_precip_mm**: Ground station measurements—most direct rain proxy.

- #### Temperature Metrics
    - **reanalysis_air_temp_k, reanalysis_avg_temp_k, reanalysis_max_air_temp_k, reanalysis_min_air_temp_k**: Reanalysis temps in Kelvin; optimal Aedes range 26-32°C accelerates larval development/virus replication.
    - **station_avg_temp_c, station_max_temp_c, station_min_temp_c**: Station temps in Celsius—ground truth validation.
    - **station_diur_temp_rng_c**: Diurnal range; wider swings stress mosquitoes.
    - **reanalysis_tdtr_k**: Temperature diurnal temperature range (reanalysis).

- #### Humidity \& Moisture
    - **reanalysis_dew_point_temp_k**: Dew point—direct humidity proxy; high values (>20°C) favor mosquito survival.
    - **reanalysis_relative_humidity_percent**: Relative humidity %—critical for egg/larval viability.
    - **reanalysis_specific_humidity_g_per_kg**: Absolute moisture content.


### For a fair fight between LightGBM and our LSTM, both models will have the same starting lineup of features. These include seasonal cues, vegetation signals, missing data hints, and early low-case periods—everything else is just how each model likes to learn over time.

### Notebook sections for the fourth project notebook (Feature Selection/Scaling/Pre-Modeling Splits)
1. Get Data
2. Redundant Feature Removal
    - Common for both models
3. Walk-forward Cross-Validation for feature importance assesment
4. Target processing
5. Benchmark model (TBC, poss notebook 04)
7. Model Tuning  (TBC)
8. Model Evaluation  (TBC, poss notebook 04)

In [1]:
import sys
import os
from pathlib import Path
from typing import List, Tuple, Any, Dict, Optional
# import gc
# import itertools
# import logging
# logging.basicConfig(level=logging.INFO)

# Set one level up as project root|
if os.path.abspath("..") not in sys.path:
    sys.path.insert(0, os.path.abspath(".."))
    
from src.config import ProjectConfig  # project config file parser
from src.utils.eda import value_streaks, top_correlations
from src.utils.visualizations import (compute_correlations_matrix,
                display_distributions, random_color, random_colormap,
                display_timeseries)

import pandas as pd
import numpy as np
import random
# import time
# from datetime import timedelta

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from src.utils.eda import top_correlations, top_vif
from src.utils.utils import _check_feature_presence, load_file, save_file
from src.preprocessing.clean import (drop_nan_rows, cap_outliers,
                                    median_groupwise_impute, pipe_clean)

from src.preprocessing.engineer.base import (reduce_features,
                                            add_missingness_features,
                                            low_value_targets)

from src.preprocessing.engineer.temporal import (circular_time_features,
                                                dynamic_temporal_features)

from src.preprocessing.engineer.pipeline import pipe_engineer

from src.preprocessing.select import remove_features

# from IPython.display import display
# import matplotlib.pyplot as plt
# import matplotlib.colors as mcolors
# from matplotlib.axis import Axis
# from matplotlib.dates import MonthLocator, YearLocator, DateFormatter
# import seaborn as sns

In [2]:
cnfg = ProjectConfig.load_configuration()
PATH_TO_RAW_DATA = cnfg.data.dirs["raw"]
PATH_TO_INTERMEDIATE_DATA = cnfg.data.dirs["intermediate"]

FILE_TRAIN_RAW= cnfg.data.files["features_train"]
FILE_TEST_RAW = cnfg.data.files["features_test"]
FILE_LABELS_RAW = cnfg.data.files["labels_train"]

PATH_TO_INTERMEDIATE_DATA = cnfg.data.dirs["intermediate"]
FILE_NAN_CLEAN = cnfg.data.files["nan_mask"]
FILE_TRAIN_CLEAN = cnfg.data.files["features_clean"]
FILE_LABELS_CLEAN = cnfg.data.files["labels_clean"]
FILE_TRAIN_ENG = cnfg.data.files["features_eng"]
FILE_LABELS_ENG = cnfg.data.files["labels_eng"]

TARGET = cnfg.preprocess.feature_groups["target"]
ENV_FEAT_PREFIX = cnfg.preprocess.feature_groups["env_prefixes"]
CITYGROUP_FEAT = cnfg.preprocess.feature_groups["city"]
WEEK_FEAT = cnfg.preprocess.feature_groups["week"]
DATETIME_FEAT = cnfg.preprocess.feature_groups["datetime"]

## Get Data

- Raw data

In [3]:
df_train_raw = load_file(path=PATH_TO_RAW_DATA / FILE_TRAIN_RAW, datetime_col=DATETIME_FEAT)
# df_test_raw = load_file(path=PATH_TO_RAW_DATA / FILE_TEST_RAW, datetime_col=DATETIME_FEAT)
df_labels_raw = load_file(path=PATH_TO_RAW_DATA / FILE_LABELS_RAW)
# list_raw_df = [df_train_raw, df_test_raw, df_labels_raw]
env_features = [f for f in df_train_raw if f.startswith(tuple(ENV_FEAT_PREFIX))]

- Enginnered data

In [4]:
# %%time
# # use pipeline output
# cleaned_data = pipe_clean(overwrite_files=True)
# df_train_clean = cleaned_data["X_clean_data"]
# df_labels_clean = cleaned_data["y_clean_data"]
# df_nan_mask = cleaned_data["nan_mask_data"]
# engineered_data = pipe_engineer(X=df_train_clean, y=df_labels_clean, overwrite_files=True)
# df_train_eng = engineered_data["X_eng_data"]
# df_labels_eng = engineered_data["y_eng_data"]

In [5]:
%%time
# or just load from disc:
# df_train_clean = load_file(path=PATH_TO_INTERMEDIATE_DATA / FILE_TRAIN_CLEAN, datetime_col=DATETIME_FEAT)
# df_labels_clean = load_file(path=PATH_TO_INTERMEDIATE_DATA / FILE_LABELS_CLEAN)
# df_nan_mask = load_file(path=PATH_TO_INTERMEDIATE_DATA / FILE_NAN_CLEAN)
df_train_eng = load_file(path=PATH_TO_INTERMEDIATE_DATA / FILE_TRAIN_ENG, datetime_col=DATETIME_FEAT)
df_labels_eng = load_file(path=PATH_TO_INTERMEDIATE_DATA / FILE_LABELS_ENG)

CPU times: user 30 ms, sys: 5.07 ms, total: 35 ms
Wall time: 34.2 ms


In [6]:
all(df_train_eng.index == df_labels_eng.index)

True

## Redundant Feature Removal

In [7]:
# # TODO remove after cleaning
# def top_vif(data: pd.DataFrame):
#     """
#     Calculate and return Variance Inflation Factor (VIF) scores for numeric features.
    
#     :param data: pandas DataFrame containing numeric and non-numeric features.
#     :return: pandas DataFrame with features and their VIF scores,
#                 sorted descending (excludes constant).
#     """
#     data_vif = add_constant(data.select_dtypes(include="number"))
#     cols = data_vif.columns
#     if data_vif.isna().sum().sum() > 1:
#         raise ValueError(f"{data_vif.isna().sum().sum()} NaNs in the dataframe.")
#     data_vif = [variance_inflation_factor(
#         data_vif.values, i) for i in range(data_vif.shape[1])]
#     data_vif = pd.DataFrame(data=data_vif, index=cols, columns=["vif"])
#     data_vif = data_vif.sort_values(by="vif", ascending=False,
#                                     na_position="first").drop(index="const")
    
#     return data_vif    

In [8]:
# def remove_features(X: pd.DataFrame, feats_to_drop: Iterable[str]=None)->pd.DataFrame:
#     """
#     Remove specified feature columns from DataFrame using config defaults or explicit list.
#     :param X: Input DataFrame.
#     :param feats_to_drop: Features to drop. If None, uses config multicollinearity removal list.
#                           Default None.
#     :return: New DataFrame copy with specified features removed. Raises ValueError if features missing.
#     """
#     if feats_to_drop is None:
#         feats_to_drop = cnfg.preprocess.multicolinear["removal_list"]
#     X_slim = X.copy()
#     if isinstance(feats_to_drop, str):
#         feats_to_drop = [feats_to_drop]
#     _check_feature_presence(target_list=feats_to_drop, source_list=X_slim.columns)
#     return X_slim.drop(columns=feats_to_drop)

In [9]:
cnfg.preprocess.multicolinear["removal_list"]

['reanalysis_sat_precip_amt_mm',
 'reanalysis_dew_point_temp_k',
 'reanalysis_air_temp_k',
 'reanalysis_specific_humidity_g_per_kg',
 'reanalysis_avg_temp_k',
 'reanalysis_max_air_temp_k',
 'reanalysis_min_air_temp_k']

In [10]:
len(df_train_eng.columns)

34

In [11]:
df_train_select = remove_features(X=df_train_eng)

In [12]:
len(df_train_select.columns)

27

In [13]:
print("Variance Inflation Factor ratios VIF 5: Feature selection after initials source features.")
temp_vif_pre = df_train_select.groupby(by=CITYGROUP_FEAT).apply(lambda group: top_vif(data=group), include_groups=False)
temp_vif_pre =  temp_vif_pre.loc["iq"].join(temp_vif_pre.loc["sj"], lsuffix="_iq", rsuffix="_sj", how="inner")
temp_vif_pre["vif_total"] = top_vif(data=df_train_select).values
temp_vif_pre.sort_values(by="vif_total", ascending=False, na_position="first")

Variance Inflation Factor ratios VIF 5: Feature selection after initials source features.


/home/butros/miniconda3/envs/py311_data_prep/lib/python3.11/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/home/butros/miniconda3/envs/py311_data_prep/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


,vif_iq,vif_sj,vif_total
missing_ndvi_s,inf,11.175960,14.816600
missing_ndvi_n,inf,13.949696,10.618118
missing_pct_env_feat,7.779426,29.729477,10.553650
missing_station,7.293415,1.827712,8.156276
reanalysis_relative_humidity_percent,6.813269,3.364903,7.034363
reanalysis_tdtr_k,6.686965,1.852951,6.249533
initial_low_case_streak,5.579551,NaN,5.789321
ndvi_south,4.568297,1.142279,5.640069
ndvi_north,4.327275,1.414127,5.414341
low_case_streak,3.910723,1.057828,5.296480


In [16]:
corr_threshold=0.8
print(f"City-stratified correlations exceeding {corr_threshold}:")
df_train_select.groupby(
    by=CITYGROUP_FEAT).apply(
        lambda group: top_correlations(
            data=group, 
            corr_threshold=corr_threshold
        ), include_groups=False)

City-stratified correlations exceeding 0.8:


city                                                               
iq    missing_ndvi_n                        missing_ndvi_s             1.000000
      missing_pct_env_feat                  missing_station            0.889715
      ndvi_north                            ndvi_south                 0.872014
      low_case_streak                       initial_low_case_streak    0.847191
      reanalysis_relative_humidity_percent  reanalysis_tdtr_k         -0.900588
sj    station_min_temp_c                    station_avg_temp_c         0.898044
      target_rolling_lag_4                  target_rolling_lag_1       0.868119
      station_avg_temp_c                    station_max_temp_c         0.864934
                                            temperature_mean_4         0.843385
      missing_ndvi_n                        missing_pct_env_feat       0.808289
      station_min_temp_c                    temperature_mean_4         0.805664
      temperature_mean_4                    sin_week

**Conclusion**:
- for engineering VIF assesments see notebook 3: `03_feature_eng_dengue.ipynb`

## Walk-forward Cross-Validation
- [ ] Test A: Baseline (clened and engineered features w/o lags and `initial_low_case_streak`)
- [ ] Test B: Baseline + lag shortlist (5 new features)
- [ ] Test C: Baseline + lag shortlist + initial_low_case_streak (IQ only)
- [ ] Test D: Baseline + lag shortlist - precip roll + initial_low_case_streak (checks what is stronger: `initial_low_case_streak` or presumably weak climate feature roll.
- [ ] Test E: Baseline + east/west aggregated vegetation `ndvi` features
- [ ] Test E: Baseline + lag shortlist - `station_avg_temp_c` to compare if high 

### Scaling
- ### RobustScaler (features)

## Target processing
- [X] Process zero/low value target value streaks
- [ ] RobustScaler (Targets)
- [ ] Predict log1p(total_cases) to reduce zero target influence (log transform targets) (cross check - complimentary or exclusive to RobustScaler)
- [ ] (Optional) LSTM specific - if LSTM results are sub-optimal, experiment with downweighting affected period data (ie 0.3 or more)

In [15]:
# DONT DO CAPPING, LOG TRANSFORM INSTEAD
# intermediate_target = cap_outliers(data=df_labels_raw, features=TARGET)
# df_labels_clean = intermediate_target["data"]
# df_labels_log = ...

# TODO: adjust **Conclusion**:
- Target outliers are 

- ### RobustScaler (targets)

# TODO Research before applying:
- **Final Preprocessing tactics for outliers:**
    - Target ("total_cases")
        - If tree models used (eg LightGBM) - no issue, trees are not sensitive to outliers:
            - [ ] use huber loss for extra safety when handling tails
            - [ ] RobustScaler may be redunndant for tree models, but if it simplifies pipeline - no harm.
            - [ ] `city` one-hot encoding (no string feature remains)
        - RNNs (eg LSTM) are outlier sensitive (gradient instability, hidden state patterns loose importance at peaks, scaling):
            - [ ] Log transform
            - [ ] Scale (RobustScaler  with IQR is more outlier resistant)
            - [ ] apply huber loss
            - [ ] groupby `city` or  one-hot encoding if passing entire dataset (no string feature remains)
            - [ ]  windowed input with:
                -  [ ] 4-8 time steps (if single step/week forecast)
    - Features:
        - [ ] RobustScaler for RNN
        - [ ] RobustScaler may be redunndant for tree models, but if it simplifies pipeline - no harm.